# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

# import libraries
import numpy as np
import pandas as pd

import sqlite3
from sqlalchemy import create_engine

import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    conn = sqlite3.connect('DisasterResponse.db')
    df = pd.read_sql('SELECT * FROM df_clean', conn)
    
    target_col_list = [col for col in df.columns if col not in ['id','message','original','genre']]

    X = df['message']
    y = df[target_col_list]
    
    return X, y

### 2. Write a tokenization function to process your text data

In [9]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [10]:
def tokenize(text):
    # replace any urls with 'urlplaceholder'
    text = re.sub(url_regex, "urlplaceholder", text)
        
    # remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 

    # convert all to lowercase
    text = text.lower()
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Lemmatize
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = WordNetLemmatizer().lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# load data
X, y = load_data()
label_list = list(y.columns)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
print(classification_report(y_test, y_pred, target_names=label_list))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5941
               request       0.81      0.58      0.68      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.78      0.67      0.72      3286
          medical_help       0.66      0.14      0.24       644
      medical_products       0.77      0.15      0.25       414
     search_and_rescue       0.93      0.06      0.11       239
              security       0.00      0.00      0.00       156
              military       0.67      0.08      0.15       267
                 water       0.83      0.47      0.60       512
                  food       0.83      0.60      0.70       878
               shelter       0.82      0.42      0.56       714
              clothing       0.75      0.12      0.21       123
                 money       0.82      0.05      0.09       192
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
def build_model(estimator, parameters):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator))
    ])

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

def display_results(cv, y_test, y_pred):
    label_list = y.columns
    report = classification_report(y_test, y_pred, target_names=label_list)
    print(report)
    print("\nBest Parameters:", cv.best_params_)


def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = build_model(estimator, parameters)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    display_results(model, y_test, y_pred)
    return(model)
                                   
def get_class_report():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = build_model(estimator, parameters)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    label_list = y.columns
    report = classification_report(y_test, y_pred, target_names=label_list, output_dict=True)                               
    return report                               
                                   
                                   
                                   

In [37]:
parameters = {
        'clf__estimator__max_iter': [100, 500],
        'clf__estimator__solver': ['liblinear', 'newton-cg', 'lbfgs']
    }
estimator = LogisticRegression()
cv = build_model(estimator, parameters)
cv.get_params()

{'cv': None,
 'error_score': 'raise',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fee14f65d08>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))],
 'estimator__vect': CountVectori

In [38]:
main()

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5941
               request       0.82      0.58      0.68      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.78      0.67      0.72      3286
          medical_help       0.66      0.15      0.24       644
      medical_products       0.77      0.15      0.25       414
     search_and_rescue       0.93      0.06      0.11       239
              security       0.00      0.00      0.00       156
              military       0.67      0.08      0.15       267
                 water       0.83      0.48      0.60       512
                  food       0.83      0.60      0.70       878
               shelter       0.82      0.42      0.56       714
              clothing       0.75      0.12      0.21       123
                 money       0.82      0.05      0.09       192
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression(max_iter = 100, solver= 'lbfgs')))
    ])

# load data
X, y = load_data()
label_list = list(y.columns)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, target_names=label_list))


                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5941
               request       0.82      0.58      0.68      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.78      0.67      0.72      3286
          medical_help       0.66      0.15      0.24       644
      medical_products       0.77      0.15      0.25       414
     search_and_rescue       0.93      0.06      0.11       239
              security       0.00      0.00      0.00       156
              military       0.67      0.08      0.15       267
                 water       0.83      0.48      0.60       512
                  food       0.83      0.60      0.70       878
               shelter       0.82      0.42      0.56       714
              clothing       0.75      0.12      0.21       123
                 money       0.82      0.05      0.09       192
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# https://www.learndatasci.com/glossary/binary-classification/
# Support vector machines
# Random forest
# Neural networks

# Run these in a loop
models = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression(max_iter = 100, solver= 'lbfgs')

# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()



def run_model():
    # load data
    X, y = load_data()
    label_list = list(y.columns)

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # train classifier
    pipeline.fit(X_train, y_train)

    # predict on test data
    y_pred = pipeline.predict(X_test)
    
    print(models[key])
    print(classification_report(y_test, y_pred, target_names=label_list))
    

for key in models.keys():
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(models[key]))
        ])
    
    run_model()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)
                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5941
               request       0.82      0.58      0.68      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.78      0.67      0.72      3286
          medical_help       0.66      0.15      0.24       644
      medical_products       0.77      0.15      0.25       414
     search_and_rescue       0.93      0.06      0.11       239
              security       0.00      0.00      0.00       156
              military       0.67      0.08      0.15       267
                 water       0.83      0.48      0.60       512
                  food       0.83 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
                        precision    recall  f1-score   support

               related       0.86      0.92      0.89      5941
               request       0.75      0.61      0.68      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.73      0.69      0.71      3286
          medical_help       0.64      0.27      0.38       644
      medical_products       0.67      0.29      0.41       414
     search_and_rescue       0.65      0.14      0.23       239
              security       0.14      0.01      0.01       156
              military       0.62      0.30      0.40       267
                 water       0.76      0.61      0.67       512
                  food       0.79      0.70      0.75       878
               s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
                        precision    recall  f1-score   support

               related       0.82      0.85      0.83      5941
               request       0.56      0.58      0.57      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.65      0.64      0.65      3286
          medical_help       0.36      0.34      0.35       644
      medical_products       0.38      0.36      0.37       414
     search_and_rescue       0.24      0.20      0.22       239
              security       0.13      0.10      0.11       156
              military       0.39      0.38      0.38       267
         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
                        precision    recall  f1-score   support

               related       0.83      0.91      0.87      5941
               request       0.66      0.60      0.63      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.65      0.66      0.65      3286
          medical_help       0.31      0.18      0.23       644
      medical_products       0.34      0.17      0.23       414
     search_and_rescue       0.47      0.07      0.12       239
              security       0.14      0.01      0.01       156
              military       0.62      0.14      0.23       267
                 water       0.41      0.36      0.38       512
                  food       0.49      0.48      0.49       878
               shelter       0.44      0.35      0.39       714
   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
# Note: the model with the highest f1-score (harmonice mean of precision and recall, 
# i.e. a single metric that weights the two metrics in a balanced way)

# SVC coming out on top so let's select that as our model and again run a grid search to determine optimum parameters

In [22]:
parameters = {
        'clf__estimator__loss': ['hinge', 'squared_hinge']
        #, 'clf__estimator__max_iter': [1000, 500, 5000],
        #'clf__estimator__dual': [True, False]
    }
estimator = LinearSVC()

cv = build_model(estimator, parameters)
cv.get_params()

{'cv': None,
 'error_score': 'raise',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f916c63f488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
        verbose=0),
              n_jobs=1))],
 'estimator__vect': CountVectorizer(analyzer='word', binary=False, decode_error='s

In [44]:
main()

                        precision    recall  f1-score   support

               related       0.86      0.93      0.89      5941
               request       0.80      0.62      0.70      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.70      0.73      3286
          medical_help       0.68      0.20      0.31       644
      medical_products       0.74      0.25      0.38       414
     search_and_rescue       0.78      0.12      0.21       239
              security       0.00      0.00      0.00       156
              military       0.69      0.25      0.36       267
                 water       0.76      0.65      0.70       512
                  food       0.79      0.75      0.77       878
               shelter       0.82      0.54      0.65       714
              clothing       0.77      0.41      0.54       123
                 money       0.80      0.12      0.22       192
        missing_people       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [46]:
estimator = LinearSVC(loss='hinge')
parameters = {
        'clf__estimator__loss': ['hinge']
        #, 'clf__estimator__max_iter': [1000, 500, 5000],
        #'clf__estimator__dual': [True, False]
    }

model = main()

                        precision    recall  f1-score   support

               related       0.86      0.93      0.89      5941
               request       0.80      0.62      0.70      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.70      0.73      3286
          medical_help       0.68      0.20      0.31       644
      medical_products       0.74      0.25      0.38       414
     search_and_rescue       0.78      0.12      0.21       239
              security       0.00      0.00      0.00       156
              military       0.69      0.25      0.36       267
                 water       0.76      0.65      0.70       512
                  food       0.79      0.75      0.77       878
               shelter       0.82      0.54      0.65       714
              clothing       0.77      0.41      0.54       123
                 money       0.80      0.12      0.22       192
        missing_people       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
filename = 'final_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.